In [1]:
import numpy as np
import metpy
from metpy import calc
from metpy.units import units
from numpy.random import seed
seed(42)
from keras.layers import Add
from tensorflow.keras import layers
import pandas as pd
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from tensorflow.keras.layers import Dropout
import keras.losses as losses
import seaborn as sns
from matplotlib import pyplot as plt
import os
import warnings  # Supress warnings
warnings.filterwarnings('ignore')
plt.rcParams['font.sans-serif'] = ['SimHei']  # 用来正常显示中文标签
plt.rcParams['axes.unicode_minus'] = False  # 用来正常显示负号
from tensorflow import keras
from tensorflow.keras import layers
os.environ["CUDA_VISIBLE_DEVICES"] = "0" # set this to the ID of the GPU you want to use
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [2]:
def get_data():
#     df = pd.read_csv('../qiancengfeng/dataset/qiancengfeng_leibao.csv')
    df = pd.read_csv('../../qiancengfeng/dataset/qiancengfeng_leibao_diag.csv')
    df_1 = pd.read_csv('../../qiancengfeng/dataset/qiancengfeng_leibao_2022.csv')
    df.time = pd.to_datetime(df['time'])
    df.stime = pd.to_datetime(df['stime'], format='ISO8601')
    df=df.sort_values(['stime','dtime']).reset_index(drop=True) 
    
    df_1.time = pd.to_datetime(df_1['time'])
    df_1.stime = pd.to_datetime(df_1['stime'], format='ISO8601')
    df_1=df_1.sort_values(['stime','dtime']).reset_index(drop=True) 
   
    df_1['time_order'] = df_1.stime.dt.hour
    
    feats_name = ['level', 'time', 'stime', 'dtime','time_order', 
            'id', 'lon', 'lat','sw_max1','sw_max2', 'v100', 'tcc', 'sf',
           'msl', 'lsp', 'cape', 'sst', 'lcc', 'sd', 'cp', 'u100', 'skt',
           'tpe', 'sp', 'd2m', 'u10', 'v10', 't2m', 'p3020', 'fg310', 'tcwv',
           'rsn', 'deg0l', 'mn2t3', 'mx2t3', 'tcw', 'fal', 'fzra', 'capes',
           '1000t', '950t', '925t', '900t', '850t', '800t', '700t',
           '600t', '500t', '400t', '300t', '250t', '200t', '150t', '100t',
           '70t', '50t', '20t', '10t', '1000gh', '950gh', '925gh', '900gh',
           '850gh', '800gh', '700gh', '600gh', '500gh', '400gh', '300gh',
           '250gh', '200gh', '150gh', '100gh', '70gh', '50gh', '20gh', '10gh',
           '1000u', '950u', '925u', '900u', '850u', '800u', '700u', '600u',
           '500u', '400u', '300u', '250u', '200u', '150u', '100u', '70u',
           '50u', '20u', '10u', '1000v', '950v', '925v', '900v', '850v',
           '800v', '700v', '600v', '500v', '400v', '300v', '250v', '200v',
           '150v', '100v', '70v', '50v', '20v', '10v', '1000r', '950r',
           '925r', '900r', '850r', '800r', '700r', '600r', '500r', '400r',
           '300r', '250r', '200r', '150r', '100r', '70r', '50r', '20r', '10r',
           '1000w', '950w', '925w', '900w', '850w', '800w', '700w', '600w',
           '500w', '400w', '300w', '250w', '200w', '150w', '100w', '70w',
           '50w', '20w', '10w', '1000q', '950q', '925q', '900q', '850q',
           '800q', '700q', '600q', '500q', '400q', '300q', '250q', '200q',
           '150q', '100q', '70q', '50q', '20q', '10q', '1000d', '950d',
           '925d', '900d', '850d', '800d', '700d', '600d', '500d', '400d',
           '300d', '250d', '200d', '150d', '100d', '70d', '50d', '20d', '10d',
           '1000pv', '950pv', '925pv', '900pv', '850pv', '800pv', '700pv',
           '600pv', '500pv', '400pv', '300pv', '250pv', '200pv', '150pv',
           '100pv', '70pv', '50pv', '20pv', '10pv']

    

    df = df[feats_name]
    df_1 = df_1[feats_name]
    df = pd.concat([df,df_1],axis=0)
    

    # 有可能有两个dtime=0,获得最一开始的
    df['previous_time'] = df['time'].shift(1)
    # 筛选符合条件的行
    condition = (df['time'] == df['previous_time']) & (df['dtime'] == 0)
    df = df[~condition]
    # 删除辅助列
    df = df.drop('previous_time', axis=1)
    return df,feats_name

# 处理异常值 *
def process_outlier(df,feats):
    _ = df[feats[7:]]
    # 1. 计算每个气象要素的平均值和标准差
    mean_values = _.mean()
    std_values = _.std()
    # 2. 根据三倍标准差法，设定异常值的阈值
    threshold = 3 * std_values
    # 3. 遍历每个气象要素的数值，将超过设定阈值的值标记为异常值
    is_outlier = (_ > mean_values + threshold) | (_ < mean_values - threshold)
    # 4. 对于标记为异常值的数据，可以根据需要选择删除、替换或进行插补处理
    # 假设你选择删除异常值
    df = df[~is_outlier.any(axis=1)]
    df = df.query('sw_max1<20 and sw_max2<20')
     
    return df

def speed_direction_features(df_):
    # 构建风向风速
    df_['wind_speed10'] = metpy.calc.wind_speed(df_['u10'].values * units('m/s'), df_['v10'].values * units('m/s'))
    df_['wind_direction10'] = metpy.calc.wind_direction(df_['u10'].values * units('m/s'),
                                                        df_['v10'].values * units('m/s'))

    df_['wind_speed100'] = metpy.calc.wind_speed(df_['u100'].values * units('m/s'),
                                                 df_['v100'].values * units('m/s'))
    df_['wind_direction100'] = metpy.calc.wind_direction(df_['u100'].values * units('m/s'),
                                                         df_['v100'].values * units('m/s'))
    
    df_['wind_speed_L1000'] = metpy.calc.wind_speed(df_['1000u'].values * units('m/s'),
                                                 df_['1000v'].values * units('m/s'))
    df_['wind_direction_L1000'] = metpy.calc.wind_direction(df_['1000u'].values * units('m/s'),
                                                         df_['1000v'].values * units('m/s'))
    
    df_['wind_speed_L950'] = metpy.calc.wind_speed(df_['950u'].values * units('m/s'),
                                                 df_['950v'].values * units('m/s'))
    df_['wind_direction_L950'] = metpy.calc.wind_direction(df_['950u'].values * units('m/s'),
                                                         df_['950v'].values * units('m/s'))
    
    df_['wind_speed_L925'] = metpy.calc.wind_speed(df_['925u'].values * units('m/s'),
                                                 df_['925v'].values * units('m/s'))
    df_['wind_direction_L925'] = metpy.calc.wind_direction(df_['925u'].values * units('m/s'),
                                                         df_['925v'].values * units('m/s'))
    
    df_['wind_speed_L900'] = metpy.calc.wind_speed(df_['900u'].values * units('m/s'),
                                                 df_['900v'].values * units('m/s'))
    df_['wind_direction_L900'] = metpy.calc.wind_direction(df_['900u'].values * units('m/s'),
                                                         df_['900v'].values * units('m/s'))
    
    
    df_['wind_direction_L500'] = metpy.calc.wind_direction(df_['500u'].values * units('m/s'),
                                                         df_['500v'].values * units('m/s'))
    
    df_['wind_direction_L700'] = metpy.calc.wind_direction(df_['700u'].values * units('m/s'),
                                                         df_['700v'].values * units('m/s'))
    return df_


def get_other_height_speed(df):
    df['wind_speed30'] = df['wind_speed10']*(30/10)**(1/7)
    df['wind_speed50'] = df['wind_speed10']*(50/10)**(1/7)
    df['wind_speed70'] = df['wind_speed10']*(70/10)**(1/7)
    return df

def get_height_speed_diff(df):
    df['wind_speed30_diff'] = df['wind_speed30']-df['wind_speed10']
    df['wind_speed50_diff'] = df['wind_speed50']-df['wind_speed30']
    df['wind_speed70_diff'] = df['wind_speed70']-df['wind_speed50']
    df['wind_speed100_diff'] = df['wind_speed100']-df['wind_speed70']
    
    df['wind_speed_L925_diff'] = df['wind_speed_L925']-df['wind_speed_L900']
    df['wind_speed_L950_diff'] = df['wind_speed_L950']-df['wind_speed_L925']
    df['wind_speed_L1000_diff'] = df['wind_speed_L1000']-df['wind_speed_L950']
    
    return df



# 构造时滞特征 *
def get_lag_features(df,ori_feats):
    df=df.sort_values(['stime','dtime']).reset_index(drop=True)
    
    
    for feat in ori_feats:
        df[feat+'_s1'] = df.groupby('stime')[feat].shift(1)
        
    # 构造后可能首尾出现空值
    df = df.fillna(method='bfill')
    df = df.fillna(method='ffill')

    return df

# 构造窗口统计特征 *
def get_window(df,ori_feats):
    df=df.sort_values(['stime','dtime']).reset_index(drop=True) 
    
    for feat in ori_feats:
        # window=8的原因是8为一个周期
        df[feat+"_max_8"] = df.groupby('stime')[feat].rolling(window=8).max().reset_index(drop=True)
        df[feat+"_avg_8"] = df.groupby('stime')[feat].rolling(window=8).mean().reset_index(drop=True)
        df[feat+"_var_8"] = df.groupby('stime')[feat].rolling(window=8).var().reset_index(drop=True)
        


    df = df.fillna(method='bfill')
    df = df.fillna(method='ffill')
    
    return df

# 时间特征 *   
def get_time_features(df):
#     df['year'] = df.time.dt.year
    df['month'] = df.time.dt.month
    df['day'] = df.time.dt.day
    df['hour'] = df.time.dt.hour
    return df

# 特征筛选 *
def select_features(df,thus):
    
    df_corr = df.drop_duplicates(subset=['time'])
    df_corr_1_res = pd.DataFrame(df_corr.corr()['sw_max1'])
    df_corr_1_res.sort_values('sw_max1',ascending=False)
    df_corr_1_res.to_csv('../../qiancengfeng/corr/sw_max1_corr.csv')
    

    df_corr_2_res = pd.DataFrame(df_corr.corr()['sw_max2'])
    df_corr_2_res.sort_values('sw_max2',ascending=False)
    df_corr_2_res.to_csv('../../qiancengfeng/corr/sw_max2_corr.csv')

    # df_corr_1_res = df_corr_1_res.query('abs(sw_max1)>=0.2')
    # df_corr_2_res = df_corr_2_res.query('abs(sw_max2)>=0.2')
    df_corr_1_res = df_corr_1_res.query('abs(sw_max1)>=@thus')
    df_corr_2_res = df_corr_2_res.query('abs(sw_max2)>=@thus')
    feats = list(set(df_corr_1_res.index).union(set(df_corr_2_res.index)))
    feats = list(filter(lambda x: x not in ['sw_max1','sw_max2','time','stime','time_order'] , feats))
    print(f'corr>{thus}:{feats}')
    
   
    feats_name = ['level', 'time', 'dtime','stime', 'id', 'lon', 'lat','type','time_order','sw_max1','sw_max2']+feats
    return feats_name

def query_cut_dataset(df,ageing,feats_name):
    df = df.set_index('stime')
    df.loc[:'2021-12-31','type'] = 'train'
    df.loc['2022-01-01':'2022-12-31','type'] = 'test'
#     df = df[:'2022']
    df = df.reset_index(drop=False)
    
    diff = 24
    if ageing==24:
        diff=25
   
    
#     df_query = df.query(f'dtime>{ageing-diff} and dtime<={ageing}').reset_index(drop=True)
    df_query = df.query(f'dtime>=0 and dtime<=120').reset_index(drop=True)
    
    df_query = df_query[feats_name]

    
    # 划分测试集-》训练集打乱后划分:去除空值然后shuffle
    TRAIN_VAL_DATA = df_query.query("type=='train'").dropna().loc[:,'sw_max1':].sample(frac=1,random_state=0)
    TRAIN_DATA = TRAIN_VAL_DATA.iloc[:int(len(TRAIN_VAL_DATA)*0.9)]
    VAL_DATA = TRAIN_VAL_DATA.iloc[int(len(TRAIN_VAL_DATA)*0.9):]
    TEST_DATA = df_query.query("type=='test'").loc[:,'sw_max1':]
    # 测试集填充
    TEST_DATA = TEST_DATA.fillna(TEST_DATA.interpolate(limit_area='inside',limit_direction='both'))
    TEST_DATA = TEST_DATA.fillna(method='bfill')
    TEST_DATA = TEST_DATA.fillna(method='ffill')
    
    return df,df_query,TRAIN_DATA,VAL_DATA,TEST_DATA


from sklearn.preprocessing import StandardScaler
def load_data(TRAIN_DATA,VAL_DATA,TEST_DATA):
    train = TRAIN_DATA
    val = VAL_DATA
    test = TEST_DATA

    scaler = StandardScaler()
    train = scaler.fit_transform(train)
    val = scaler.transform(val)
    test = scaler.transform(test)
    data = dict()

    data["train_y"] = train[:, :2]
    data["train_X"] = train[:, 2:]
    data["val_y"] = val[:, :2]
    data["val_X"] = val[:, 2:]
    data["test_y"] = test[:, :2]
    data["test_X"] = test[:, 2:]
    data["scaler"] = scaler
    print(scaler.mean_)
    return data

def quantile_loss(y_true, y_pred, q):
        
    error = y_true - y_pred
    return tf.reduce_mean(tf.maximum(q * error, (q - 1) * error))


from tensorflow.keras.layers import Dense, Add, Input, LayerNormalization
from tensorflow.keras.models import Model

def build_network(input_features=None, num_blocks=3):
    inputs = Input(shape=(input_features,), name="input")
    x = Dense(512, activation='relu', name="hidden_1")(inputs)
    
    for i in range(num_blocks):
        residual = x
        x = Dense(1024, activation='relu', name=f"hidden_{i+2}_1")(x)
        x = Dense(512, activation='linear', name=f"hidden_{i+2}_2")(x)
        x = Dropout(0.3, name=f"dropout_{i+2}")(x)  # 添加Dropout层
        x = Add()([x, residual])
        
    prediction = Dense(2, activation='linear', name="final")(x)

    model = Model(inputs=inputs, outputs=prediction)
    opt = keras.optimizers.Adam(learning_rate=0.0001)
     # 定义百分位值
#     q = 0.7
    q = 0.7  # 设置为0.9，即求解90%百分位损失

    # 编译模型时使用百分位损失
    model.compile(optimizer=opt, loss=lambda y_true, y_pred: quantile_loss(y_true, y_pred, q))
#     model.compile(optimizer=opt, loss="mae")

    return model


def lr_scheduler(epoch, lr):
    if epoch % 10 == 0 and epoch > 0:
        lr = lr * 0.1
    return lr

def train_model():

    input_features = data["train_X"].shape[1]
    model = build_network(input_features=input_features)
    print("Network Structure")
    print(model.summary())
    print("Training Data Shape: " + str(data["train_X"].shape))



    lr_callback = keras.callbacks.LearningRateScheduler(lr_scheduler)
    es_callback = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)
    history  =  model.fit(x=data["train_X"], y=data["train_y"], batch_size=32, epochs=2000,verbose=1,callbacks=[es_callback,lr_callback],
              validation_data=(data["val_X"], data["val_y"]))

    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'], label='val_loss')
    plt.title('Model Loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.show()
    return model

from pykalman import KalmanFilter
import pandas as pd
import numpy as np
def kalman_sm_max(target):
    
    kalman_res = pd.DataFrame()

    # 对每个起始时间应用卡尔曼滤波器
    for stime, group in df_res.groupby('stime'):
        # 按照dtime排序，保证预报按照时间顺序
        group = group.sort_values('dtime')

        predicted_sw_max = group['nn_'+target].values  # 获取当前时间序列的神经网络预测的温度值

        initial_state = predicted_sw_max[0]  # 初始状态设为神经网络预测的第一个温度值
        filtered_state_means = np.zeros(predicted_sw_max.shape)  # 初始化一个用于存放滤波状态均值的数组
        filtered_state_covariances = np.zeros(predicted_sw_max.shape)  # 初始化一个用于存放滤波状态协方差的数组
        global_error_variance = np.var(predicted_sw_max - filtered_state_means)  # 计算一个全局误差方差

        kf = KalmanFilter(transition_matrices=[1],
                          observation_matrices=[1],
                          initial_state_mean=initial_state,
                          initial_state_covariance=1,
                          transition_covariance=np.var(predicted_sw_max))

        for t in range(len(predicted_sw_max)):
            if t == 0:
                # 第一个点，用神经网络初始预测值
                filtered_state_means[t] = initial_state
                filtered_state_covariances[t] = 1
            else:
                # 更新卡尔曼滤波
                filtered_state_means[t], filtered_state_covariances[t] = kf.filter_update(
                    filtered_state_mean=filtered_state_means[t-1],
                    filtered_state_covariance=filtered_state_covariances[t-1],
                    observation=predicted_sw_max[t],
                    observation_matrix=np.atleast_2d(1),
                    observation_covariance=0.5*global_error_variance
                )

        # 将处理后的预报添加到新的DataFrame
        group['kalman_'+target] = filtered_state_means
        kalman_res = pd.concat([kalman_res, group])
    return kalman_res

In [3]:
df,feats_name = get_data()  # 获得数据

In [4]:
df = process_outlier(df,feats_name)  # 处理异常值

In [5]:
df.describe()

,level,time,stime,dtime,time_order,id,lon,lat,sw_max1,sw_max2,...,400pv,300pv,250pv,200pv,150pv,100pv,70pv,50pv,20pv,10pv
count,78457.0,78457,78457,78457.000000,78457.000000,78457.0,7.845700e+04,7.845700e+04,78457.000000,78457.000000,...,7.845000e+04,7.845000e+04,7.845000e+04,7.845000e+04,7.845000e+04,78450.000000,78450.000000,78450.000000,78450.000000,78450.000000
mean,0.0,2020-01-14 12:07:01.774985216,2020-01-12 02:56:41.639114752,57.172260,5.365181,2.0,1.020300e+02,2.823000e+01,2.909475,3.283945,...,4.777314e-07,5.146018e-07,6.076840e-07,7.473594e-07,1.018405e-06,0.000003,0.000014,0.000024,0.000067,0.000171
min,0.0,2017-01-01 15:00:00,2017-01-01 00:00:00,0.000000,0.000000,2.0,1.020300e+02,2.823000e+01,0.000000,0.000000,...,-6.805687e-06,-5.110219e-06,-9.173519e-06,-8.132207e-06,-3.017080e-06,-0.000033,-0.000021,-0.000015,0.000015,-0.000008
25%,0.0,2018-08-13 18:00:00,2018-08-10 12:00:00,24.000000,0.000000,2.0,1.020300e+02,2.823000e+01,1.500000,2.000000,...,1.846909e-07,1.163694e-07,8.694451e-08,1.027461e-07,3.118676e-07,0.000002,0.000010,0.000020,0.000058,0.000143
50%,0.0,2020-02-17 12:00:00,2020-02-16 00:00:00,51.000000,0.000000,2.0,1.020300e+02,2.823000e+01,2.600000,3.000000,...,4.234976e-07,3.454252e-07,3.226519e-07,3.774367e-07,7.283884e-07,0.000003,0.000013,0.000023,0.000067,0.000171
75%,0.0,2021-05-26 00:00:00,2021-05-23 12:00:00,84.000000,12.000000,2.0,1.020300e+02,2.823000e+01,4.000000,4.000000,...,7.072759e-07,7.261644e-07,7.965073e-07,9.650466e-07,1.418186e-06,0.000004,0.000016,0.000027,0.000076,0.000200
max,0.0,2022-12-31 21:00:00,2022-12-31 00:00:00,144.000000,12.000000,2.0,1.020300e+02,2.823000e+01,15.300000,19.900000,...,9.684358e-06,1.005564e-05,1.597344e-05,1.914169e-05,5.403170e-06,0.000046,0.000050,0.000066,0.000120,0.000360
std,0.0,NaN,NaN,39.866818,5.966361,0.0,1.421095e-14,1.065821e-14,1.775539,1.786028,...,5.502051e-07,7.263126e-07,9.464603e-07,1.143633e-06,1.097956e-06,0.000002,0.000007,0.000008,0.000015,0.000048


In [6]:
df = speed_direction_features(df) # 合成风向、风速
df = get_other_height_speed(df)   # 计算其他高度层的风速

df = get_height_speed_diff(df) # 相邻高度风速差
df = get_time_features(df) # 加入时间特征
ori_feats = ['wind_speed10','wind_speed30','wind_speed50','wind_speed70','wind_speed100','wind_speed_L900','wind_speed_L925','wind_speed_L950','wind_speed_L1000']
df = get_window(df,ori_feats)
df = get_lag_features(df,ori_feats) # 时滞特征无效
# thus=0
feats_name = select_features(df,0.2) # 特征筛选

corr>0.2:['skt', '700r', '950w', 'wind_speed70_var_8', '900v', 'wind_speed70_avg_8', '800w', '925r', '1000q', '925w', '900r', '900w', '500gh', '600gh', 'fg310', '850w', '925u', 'wind_speed100_max_8', 'wind_speed50_var_8', 'tcw', 'wind_speed50_diff', 'wind_speed_L925_max_8', '400gh', '1000r', 'v10', '925v', 'lsp', 'd2m', '950u', 'wind_speed_L900_avg_8', 'v100', 'wind_speed_L925_avg_8', 'tcwv', '200gh', 'wind_speed10_max_8', '950r', 'wind_speed_L1000_s1', '800q', '800v', '50gh', 'wind_speed_L1000_max_8', 'wind_speed_L1000', 'wind_speed30_diff', 'wind_speed_L950', 'sp', 'wind_speed50_avg_8', '925q', '250gh', 'month', 'lcc', 'wind_speed_L900_max_8', '700gh', '20q', 'wind_speed_L950_avg_8', 'wind_speed30', 'wind_speed_L1000_avg_8', 'wind_speed30_avg_8', '700q', 'wind_speed10_var_8', '850r', '10q', '950v', '900u', 'wind_speed_L950_s1', '950q', 'wind_speed30_max_8', '150gh', 'wind_speed70_max_8', '300t', 'wind_speed70_diff', 'wind_speed50_max_8', 'wind_speed30_var_8', '1000v', 'u10', '1000w',

In [10]:
df.query("type=='train'").describe()

,stime,level,time,dtime,time_order,id,lon,lat,sw_max1,sw_max2,...,wind_speed_L1000_var_8,wind_speed10_s1,wind_speed30_s1,wind_speed50_s1,wind_speed70_s1,wind_speed100_s1,wind_speed_L900_s1,wind_speed_L925_s1,wind_speed_L950_s1,wind_speed_L1000_s1
count,70608,70608.0,70608,70608.000000,70608.000000,70608.0,7.060800e+04,7.060800e+04,70608.000000,70608.000000,...,70608.000000,67654.000000,67654.000000,67654.000000,67654.000000,67654.000000,67654.000000,67654.000000,67654.000000,67654.000000
mean,2019-10-02 12:04:46.335826176,0.0,2019-10-04 21:24:15.081577472,57.324652,5.961591,2.0,1.020300e+02,2.823000e+01,3.058811,3.293768,...,0.190751,1.050654,1.229193,1.322247,1.387356,1.543049,0.873881,0.875159,0.872559,0.873663
min,2017-01-01 00:00:00,0.0,2017-01-01 15:00:00,0.000000,0.000000,2.0,1.020300e+02,2.823000e+01,0.000000,0.000000,...,0.002142,0.004081,0.004775,0.005136,0.005389,0.007878,0.003735,0.002310,0.004197,0.001728
25%,2018-06-20 00:00:00,0.0,2018-06-21 14:15:00,24.000000,0.000000,2.0,1.020300e+02,2.823000e+01,1.900000,2.000000,...,0.092175,0.621479,0.727087,0.782130,0.820644,0.805020,0.481682,0.485367,0.482851,0.481798
50%,2019-10-25 12:00:00,0.0,2019-10-28 00:00:00,51.000000,0.000000,2.0,1.020300e+02,2.823000e+01,3.000000,3.000000,...,0.164302,0.931201,1.089441,1.171916,1.229623,1.367877,0.799097,0.799687,0.797067,0.798635
75%,2021-02-07 12:00:00,0.0,2021-02-10 06:00:00,84.000000,12.000000,2.0,1.020300e+02,2.823000e+01,4.000000,4.000000,...,0.259649,1.348329,1.577452,1.696871,1.780427,2.085095,1.180896,1.179869,1.176690,1.178622
max,2021-12-31 12:00:00,0.0,2021-12-31 21:00:00,144.000000,12.000000,2.0,1.020300e+02,2.823000e+01,15.300000,19.900000,...,0.912768,4.352827,5.092506,5.478028,5.747774,6.728014,3.177345,3.236651,3.229226,3.193826
std,NaN,0.0,NaN,39.947706,5.999920,0.0,1.421096e-14,1.065822e-14,1.789434,1.793070,...,0.129094,0.618779,0.723929,0.778733,0.817079,0.961619,0.499460,0.498596,0.497191,0.497793


In [8]:
test_pred={}
for ageing in [24,48,72,96,120]:
    df,df_query,TRAIN_DATA,VAL_DATA,TEST_DATA = query_cut_dataset(df,ageing,feats_name) # 查询和切分数据
    data = load_data(TRAIN_DATA,VAL_DATA,TEST_DATA)
    print("-"*40)
    print('Shape of Data',df.shape)
    print('Ageing',ageing)
    print("-"*40)
   
    model = train_model()
 
    df_res = df_query.query("type=='test'").loc[:,:'sw_max2'].reset_index(drop=True)
    y_hat = model.predict(data["test_X"])
    # 反归一化
    y_hat = data["scaler"].inverse_transform(np.concatenate((y_hat,data["test_X"]),axis=1))[:,:2]
    # 神经网络
    df_res['nn_sw_max1'] = y_hat[:,0]
    df_res['nn_sw_max2'] = y_hat[:,1]
    print('df_res.isnull().sum()',df_res.isnull().sum().sum())
    # kalman
    df_res = kalman_sm_max('sw_max1')
    df_res = kalman_sm_max('sw_max2')
    # 平均
    df_res['ensemble_sw_max1'] = (df_res['nn_sw_max1']+df_res['kalman_sw_max1'])/2
    df_res['ensemble_sw_max2'] = (df_res['nn_sw_max2']+df_res['kalman_sw_max2'])/2
    test_pred[ageing] = df_res
    break

[ 3.06044293e+00  3.30086337e+00  2.86801472e+02  8.11264930e+01
 -1.84631317e-02  5.33536530e-01 -5.26189423e-02  1.38367123e+00
 -2.39929215e-02  7.68964975e+01  9.20660259e-03 -1.97236029e-02
  7.68955845e+01 -2.02937999e-02  5.81656130e+03  4.36717982e+03
  6.06004930e+00 -2.29121553e-02 -1.20136621e-01  2.79697144e+00
  4.84633346e-01  2.12981985e+01  9.29015309e-02  1.55322752e+00
  7.52818323e+03  7.68753699e+01 -8.75286196e-02 -5.25310755e-02
  3.86738827e-03  2.81672271e+02 -1.13644009e-01  8.74250206e-01
 -1.42699298e-01  8.75601566e-01  2.11088470e+01  1.23851466e+04
  1.92971732e+00  7.68930170e+01  8.72002343e-01  9.20658585e-03
 -7.74967054e-02  2.06788261e+04  1.54992182e+00  8.74465845e-01
  1.78245055e-01  8.73363162e-01  7.80353125e+04  1.31873483e+00
  9.20659702e-03  1.08967229e+04  7.25253478e+00  4.01896244e-01
  1.55076866e+00  3.10755870e+03  2.99029065e-06  8.72948067e-01
  1.22717228e+00  8.74097373e-01  1.22592753e+00  8.12508293e-03
  3.05990242e-01  7.68868

KeyboardInterrupt: 

In [ ]:
test_pred

In [ ]:
# # 画图分析相关性
# df_draw = df.query("type=='test'").drop_duplicates(subset=['time'])
# df_draw = df_draw.set_index('time')

# df_draw = df_draw.sort_index()

# import matplotlib.pyplot as plt
# plt.rcParams['font.size'] = 15

# df_draw[['sw_max1','sw_max2']] = df_draw[['sw_max1','sw_max2']]

# df_draw[['wind_speed100','wind_speed_L925','wind_speed10','sw_max1','sw_max2']][:130].dropna().plot(figsize=(20,10))
# df_draw = df_draw[['wind_speed100','wind_speed_L925','wind_speed10','sw_max1','sw_max2']]
# plt.title('最大浅层风、10米风速、100米风速可视化')
# legend_labels = {'wind_speed100': '100米风速', 'wind_speed10': '10米风速', 'sw_max1': '最大浅层风1', 'sw_max2': '最大浅层风2'}
# plt.legend(labels=legend_labels.values())
# plt.xlabel('时间(间隔3h)')
# plt.ylabel('米/秒')
# plt.show()

# df_corr = pd.DataFrame(df_draw.corr()['sw_max2'])
# df_corr = df_corr.sort_values('sw_max2',ascending=False)
# # df_corr_2 = df_corr.query('abs(sw_max2)>=0.1')
# df_corr

In [ ]:

# model.save("model/swallow_wind_model_0.h5")

# 推理阶段

In [ ]:
test_pred.keys()

In [ ]:
def res_metric(df_res):
    for ageing in test_pred.keys():
        for name in ['nn_','kalman_','ensemble_']: 
            for sw_max in ['sw_max1','sw_max2']:
                sw_max_mse = mean_squared_error(df_res[name+sw_max], df_res[sw_max])
                sw_max_mae = mean_absolute_error(df_res[name+sw_max], df_res[sw_max])
                print(str(ageing)+'_'+name+sw_max,sw_max_mse,sw_max_mae)


In [ ]:
concated_res = test_pred[24]
# for ageing in [48,72,96,120]:
#     concated_res = pd.concat([concated_res,test_pred[ageing]],axis=0)

In [ ]:
res_metric(concated_res)

In [ ]:
concated_res

In [ ]:
left='2022-12-01 00:00:00'
right='2022-12-30 00:00:00'

concated_res.query("dtime<=24").set_index('time').sort_index()[['nn_sw_max1','sw_max1']][left:right].plot()
concated_res.query("dtime>24 and dtime<=48").set_index('time').sort_index()[['nn_sw_max1','sw_max1']][left:right].plot()
concated_res.query("dtime>48 and dtime<=72").set_index('time').sort_index()[['nn_sw_max1','sw_max1']][left:right].plot()
concated_res.query("dtime>72 and dtime<=96").set_index('time').sort_index()[['nn_sw_max1','sw_max1']][left:right].plot()
concated_res.query("dtime>96 and dtime<=120").set_index('time').sort_index()[['nn_sw_max1','sw_max1']][left:right].plot()

concated_res.query("dtime<=24").set_index('time').sort_index()[['nn_sw_max2','sw_max2']][left:right].plot()
concated_res.query("dtime>24 and dtime<=48").set_index('time').sort_index()[['nn_sw_max2','sw_max2']][left:right].plot()
concated_res.query("dtime>48 and dtime<=72").set_index('time').sort_index()[['nn_sw_max2','sw_max2']][left:right].plot()
concated_res.query("dtime>72 and dtime<=96").set_index('time').sort_index()[['nn_sw_max2','sw_max2']][left:right].plot()
concated_res.query("dtime>96 and dtime<=120").set_index('time').sort_index()[['nn_sw_max2','sw_max2']][left:right].plot()
plt.show()

In [ ]:
concated_res.query("dtime<=24").set_index('time').sort_index()[['nn_sw_max1','sw_max1']]['2022-03-13':'2022-03-17']

## 保存

In [ ]:
import datetime
_time = datetime.datetime.now().strftime('%Y%m%d%H%M')

In [ ]:
concated_res.to_csv(f'../../qiancengfeng/result/swallow_wind_{_time}.csv',index=False)

## 数据评测

In [ ]:
# plt.title("Predicted Distribution vs. Actual")

# sns.distplot(df_res_not_null['sw_max_pred_nn'], label="y_hat")
# sns.distplot(df_res_not_null['sw_max_true'], label="y_true")
# plt.legend()
# # plt.savefig("pred_dist.jpg")
# plt.show()

In [ ]:
df_res

## 效果可视化

In [ ]:
df_res = df_res.set_index('stime')

In [ ]:
df_res[['sw_max_pred_nn','sw_max_true']]['2022-01-16':'2022-01-20'].plot(figsize=(20,10))
plt.show()

In [ ]:
# df_res.to_csv('../qiancengfeng/result/swallow_wind_time_order_12.csv',index=False)

## 统计每个时刻的指标

In [ ]:
# def metric_by_attribute(df_res,attribute,true,pred):
#     res_dic={}
#     for k,v in df_res.groupby(attribute):
#     #     rmse = np.sqrt(((v['true']-v['pred'])**2).sum()/len(v))
#         rmse = np.sqrt(np.mean((v[true]-v[pred])**2))
#         mae = np.mean(np.abs(v[true]-v[pred]))
#         res_dic.update({k:[rmse,mae]})

#     res = np.array(list(res_dic.values()))

#     # 获取横坐标和纵坐标的值
#     x_values = list(res_dic.keys())
#     res_rmse = res[:,0]
#     res_mae = res[:,1]
#     # 绘制折线图
#     plt.plot(x_values, res_rmse,marker='*',label='rmse')
#     # 设置横坐标和纵坐标的标签
#     plt.xlabel(attribute)
#     plt.ylabel('rmse/mae')
#     # 显示图形
#     plt.plot(x_values, res_mae,marker='+',label='mae')
#     plt.legend()
#     # 设置横坐标和纵坐标的标签
#     # 显示图形
#     plt.show()

# metric_by_attribute(df_res.dropna(),'dtime','sw_max_true','sw_max_pred_nn')

# ## 统计每个区间的指标

# # df_res['true'].hist()
# # plt.show()

# bins = [0, 2, 4, 6, 12,999]
# labels = ['0-2', '2-4', '4-6', '8-12','>12']
# # 使用cut函数进行分箱
# df_res['sw_max_true_bin'] = pd.cut(df_res['sw_max_true'], bins,labels=labels)

# metric_by_attribute(df_res.dropna(),'sw_max_true_bin','sw_max_true','sw_max_pred_nn')

# df_res